# 교차 검증과 그리드 서치

### 교차 검증의 필요성
좋은 모델을 만들기위해서 하이퍼 파라미터를 튜닝할 필요가 있다. 이때, 테스트 세트를 기준으로 평가하면 다음과 같은 문제가 있다.
- 일반화 능력의 과대평가 : 개발된 모델은 일반적인 데이터에 대해 잘 작동할것으로 기대된다. 하지만 반복적으로 테스트세트를 사용하여 개발하면, 테스트 세트에 특화된 모델이 개발될 수 있다.
- 과적합 : 테스트 세트에 과적합된 모델이 개발될 수 있다.
- 평가의 객관적 상실

그래서 테스트 세트는 모델을 최종적으로 평가할 때만 사용해야한다.

### 검증 세트
교차검증에서는 트레인 세트의 일부를 검증 세트로 사용한다.

5-fold 교차검증에서는 트레인 세트를 5분할하여, 검증세트를 바꿔가며 교차검증을 하게 된다.


In [1]:
import pandas as pd
import numpy as np

wine = pd.read_csv('https://bit.ly/wine_csv_data')

display(wine.head())
wine['class'].value_counts()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


class
1.0    4898
0.0    1599
Name: count, dtype: int64

In [2]:
feature = wine.iloc[:, :3]
target = wine.iloc[:, 3]

In [55]:
# 훈련세트와 테스트 세트 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(feature, target, random_state=42, test_size=0.2)

# 훈련세트에서 검증세트 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state=42, test_size=0.2)


In [56]:
# 검증세트를 사용해 모델을 평가.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차검증

사이킷런에는 교차검증 함수가 있다.

In [57]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)  # 기본적으로 5-폴드 교차검증
print(f"모델 훈련 시간 : {scores['fit_time']}")
print(f"모델 검증 시간 : {scores['score_time']}")
print(f"모델 점수 : {scores['test_score']}\n평균(검증 폴드의 점수) : {np.mean(scores['test_score'])}")

모델 훈련 시간 : [0.00853181 0.00751114 0.00699759 0.00553632 0.0061667 ]
모델 검증 시간 : [0.0033288  0.00462222 0.00199986 0.0030036  0.00300026]
모델 점수 : [0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]
평균(검증 폴드의 점수) : 0.855300214703487


10-폴드 교차검증을 수행하려면 다음과 같이 작성한다.

In [58]:
from sklearn.model_selection import StratifiedKFold # Stratified : 계층화된. 하위 그룹으로 나누느것.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  # 교차검증을 위한 분할기 생성. 10분할, 분할 하기전에 데이터세트를 셔플.
scores = cross_validate(dt, train_input, train_target, cv=splitter)  # cross-validation
print(f"모델 훈련 시간 : {scores['fit_time']}")
print(f"모델 검증 시간 : {scores['score_time']}")
print(f"모델 점수 : {scores['test_score']}\n평균(검증 폴드의 점수) : {np.mean(scores['test_score'])}")

모델 훈련 시간 : [0.01254797 0.01053548 0.00951195 0.00903034 0.00600123 0.00699782
 0.01050925 0.00850797 0.00661182 0.01188564]
모델 검증 시간 : [0.00300026 0.00500274 0.00350904 0.00248241 0.0025847  0.00252867
 0.00350666 0.00199771 0.00200033 0.00099945]
모델 점수 : [0.83461538 0.87884615 0.85384615 0.85384615 0.84615385 0.87307692
 0.85961538 0.85549133 0.85163776 0.86705202]
평균(검증 폴드의 점수) : 0.8574181117533719


### 하이퍼파라미터 튜닝
좋은 모델을 얻기 위해서 하이퍼파라미터를 튜닝한다. 여러 매개변수를 전부 튜닝하면서 최적의 하이퍼파라미터를 찾는것은 복잡하다. 

**그리드서치**
사이킷런에서 제공한다. 하이퍼파라미터 탐색과 교차검증을 한번에지원한다.

In [61]:
from sklearn.model_selection import GridSearchCV  # 가능한 조합의 교차를 그리드로 생각하여 선택
# 탐색할 하이퍼 파라미터를 딕셔너리 형태로. 여기선 5개의 값중에서 탐색
params = {'min_impurity_decrease':  # min_impurity_decrease : 분할시 정보이득이 이 값이 이하이면 분할하지 않음. 기본값은 0
          [i * 0.0001 for i in range(1, 6)]}

# 그리드 서치 객체를 만들때 모델과 탐색할 하이퍼 파라미터를 넣는다.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=params, n_jobs=-1)  # n_jobs는 병렬 실행에 사용할 cpu 코어수

# 그리드 서치 객체는 brute force 방식으로 하이퍼 파라미터를 찾으며, 기본적으로 교차검증은 5번이다.
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002,
                                                   0.00030000000000000003,
                                                   0.0004, 0.0005]})

In [71]:
dt = gs.best_estimator_
params = dt.get_params()

# print(params['min_impurity_decrease'])  # 선택된 최적의 파라미터 확인
print(gs.best_params_)

print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

{'min_impurity_decrease': 0.0001}
0.9615162593804117
0.8653846153846154


In [73]:
print(gs.cv_results_['mean_test_score']) # 교차 검증 평균 점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


더 복잡한 매개변수 조합으로 그리드서치

In [80]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.00005),
          'max_depth' : range(5, 30, 1),            # 트리의 최대 깊이. 깊을수록 모델이 복잡해짐(과적합)
          'min_samples_split' : range(2, 200, 10)   # 나눌때의 최소 샘플 수. 작을수록 모델이 복잡해짐.   
}

In [81]:
# 모델 훈련
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), param_grid=params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 30),
                         'min_impurity_decrease': array([0.0001 , 0.00015, 0.0002 , 0.00025, 0.0003 , 0.00035, 0.0004 ,
       0.00045, 0.0005 , 0.00055, 0.0006 , 0.00065, 0.0007 , 0.00075,
       0.0008 , 0.00085, 0.0009 , 0.00095]),
                         'min_samples_split': range(2, 200, 10)})

In [82]:
# 결과 확인
print(gs.best_params_)
print(gs.best_score_)

{'max_depth': 14, 'min_impurity_decrease': 0.00045000000000000004, 'min_samples_split': 12}
0.8697336566224921


### 랜덤 서치
그리드 서치는 브루트 포스 방식이기 때문에 시간이 많이 걸리며, 매개변수의 범위를 지정하기도 어렵다. 이럴 때 랜덤 서치를 사용하면 좋다.

랜덤 서치에서는 매개변수를 샘플링할 수 있는 확률분포 객체를 전달한다.

In [83]:
from scipy import stats

In [103]:
rgen = stats.randint(0, 10)  # 객체 생성
print(rgen.rvs(10))          # 객체에서 난수 생성
ugen = stats.uniform(0, 1)
print(ugen.rvs(10))


[6 7 1 7 2 5 2 6 1 4]
[0.71039567 0.64900731 0.59795144 0.44283401 0.1298677  0.93687251
 0.89463848 0.05990805 0.55414367 0.05394556]


In [115]:
from sklearn.model_selection import RandomizedSearchCV
params = {'min_impurity_decrease' : stats.uniform(0.0001, 0.001),
          'max_depth' : stats.randint(10, 50),            # 트리의 최대 깊이. 깊을수록 모델이 복잡해짐(과적합)
          'min_samples_split' : stats.randint(2, 50),   # 나눌때의 최소 샘플 수. 작을수록 모델이 복잡해짐.   
          'min_samples_leaf' : stats.randint(1, 50)
}
rs = RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42), random_state=42, param_distributions=params, n_iter=1200, n_jobs = -1)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=1200, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F92566B790>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F9258AED10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F926245310>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F9262467D0>},
                   random_state=42)

In [117]:
rs.best_params_

{'max_depth': 41,
 'min_impurity_decrease': 0.0004403871042545057,
 'min_samples_leaf': 4,
 'min_samples_split': 6}

In [119]:
print(rs.best_score_)
dt =rs.best_estimator_
print(dt.score(test_input, test_target))

0.8701182720071075
0.8607692307692307
